# Calibration dataset

In [ ]:
import datasets
from tqdm import tqdm

In [ ]:
subset_size = 300
calibration_data = []
# dataset = datasets.load_dataset("Nahrawy/VIDIT-Depth-ControlNet", split="train", streaming=True).shuffle(seed=42).take(subset_size)
dataset = datasets.load_dataset("depth-anything/DA-2K", split="train", streaming=True).shuffle(seed=42).take(subset_size)
for batch in tqdm(dataset):
    image = np.array(batch["image"])[...,:3]
    input_tensor, _ = image_preprocess(image)
    calibration_data.append(input_tensor)

# Quantization

In [ ]:
# input model
ov_model_path = "models_ov/depth_anything_v2_vitl.xml"
# output path
ov_model_int8_path = ov_model_path.replace(".xml", "_int8.xml")

In [ ]:
import nncf
import openvino as ov

core = ov.Core()

model = core.read_model(ov_model_path)
quantized_model = nncf.quantize(
    model=model,
    subset_size=subset_size,
    model_type=nncf.ModelType.TRANSFORMER,
    calibration_dataset=nncf.Dataset(calibration_data),
)
ov.save_model(quantized_model, ov_model_int8_path)